Índices invertidos e busca booleana
===================================

_Flávio Codeço Coelho (Com a contribuição dos alunos do curso de Sistemas de recuperação de Informações da EMAp)_

Nesta prática, vamos contruir um indice invertido e uma máquina de busca booleana simples.

Para agilizar nosso trabalho, vamos utilizar a biblioteca [NLTK](http://nltk.org) para processamento de linguagem natural.

In [17]:
import nltk
import os

Em seguida vamos importar mais coisas necessárias para o nosso trabalho. Note que estamos baixando a obra completa de Machado de Assis, com a qual iremos alimentar nosso índice.

In [2]:
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer

Vamos também baixar o banco de *stopwords* do NLTK. Stop words são um conjunto de palavras que normalmente carregam baixo conteúdo semântico e portanto não são alvo de buscas.

In [3]:
nltk.download('stopwords')
nltk.download('machado')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fccoelho\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package machado to
[nltk_data]     C:\Users\fccoelho\AppData\Roaming\nltk_data...


True

In [21]:
textos = []
for p, d, f in os.walk(r'machado\machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid),'r') as g:
                textos.append(g.read())
                

In [22]:
textos[0]

"Conto, Contos Fluminenses, 1870\n\nContos Fluminenses\n\nTexto-fonte:\n\nObra Completa, Machado de Assis, vol. II,\n\nRio de Janeiro: Nova Aguilar, 1994.\n\nPublicado originalmente pela\nEditora Garnier, Rio de Janeiro, em 1870.\n\nÍNDICE\n\nMISS DOLLAR\n\nLUÍS\nSOARES\n\nA MULHER DE\nPRETO\n\nO\nSEGREDO DE AUGUSTA\n\nCONFISSÕES DE UMA VIÚVA MOÇA\n\nLINHA\nRETA E LINHA CURVA\n\nFREI\nSIMÃO\n\nMISS\nDOLLAR\n\nÍNDICE\n\nCapítulo Primeiro\n\nCapítulo II\n\nCapítulo iii\n\nCapítulo iv\n\nCapítulo v\n\nCapítulo vI\n\nCapítulo vII\n\nCAPÍTULO VIII\n\nCAPÍTULO PRIMEIRO\n\nEra conveniente ao romance que o leitor\nficasse muito tempo sem saber quem era Miss Dollar. Mas por outro lado,\nsem a apresentação de Miss Dollar, seria o autor obrigado a longas\ndigressões, que encheriam o papel sem adiantar a ação. Não há hesitação\npossível: vou apresentar-lhes Miss Dollar.\n\nSe o leitor é rapaz e dado ao gênio\nmelancólico, imagina que Miss Dollar é uma inglesa pálida e delgada,\nescassa de carnes e

Lendo o texto *puro* dos livros de Machado:

In [23]:
#textos = [machado.raw(id) for id in machado.fileids()]
#len(textos)

Carregando a  lista de stopwords em lingua portuguesa para limpeza dos textos. Note que é preciso trazer as palavras para *UTF-8* antes de usá-las.

In [24]:
swu = stopwords.words('portuguese') + list (string.punctuation)
#swu = [word.decode('utf8') for word in sw]

In [25]:
stopwords.words('portuguese')
#list (string.punctuation)

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera',
 'es

Um outro ingrediente essencial é um stemmer para a nossa língua. O Stemmer reduz as palavras a uma abreviação que se aproxima da "raiz" da palavra.

In [26]:
stemmer = PortugueseStemmer()

In [27]:
WordPunctTokenizer().tokenize(textos[0])

['Conto',
 ',',
 'Contos',
 'Fluminenses',
 ',',
 '1870',
 'Contos',
 'Fluminenses',
 'Texto',
 '-',
 'fonte',
 ':',
 'Obra',
 'Completa',
 ',',
 'Machado',
 'de',
 'Assis',
 ',',
 'vol',
 '.',
 'II',
 ',',
 'Rio',
 'de',
 'Janeiro',
 ':',
 'Nova',
 'Aguilar',
 ',',
 '1994',
 '.',
 'Publicado',
 'originalmente',
 'pela',
 'Editora',
 'Garnier',
 ',',
 'Rio',
 'de',
 'Janeiro',
 ',',
 'em',
 '1870',
 '.',
 'ÍNDICE',
 'MISS',
 'DOLLAR',
 'LUÍS',
 'SOARES',
 'A',
 'MULHER',
 'DE',
 'PRETO',
 'O',
 'SEGREDO',
 'DE',
 'AUGUSTA',
 'CONFISSÕES',
 'DE',
 'UMA',
 'VIÚVA',
 'MOÇA',
 'LINHA',
 'RETA',
 'E',
 'LINHA',
 'CURVA',
 'FREI',
 'SIMÃO',
 'MISS',
 'DOLLAR',
 'ÍNDICE',
 'Capítulo',
 'Primeiro',
 'Capítulo',
 'II',
 'Capítulo',
 'iii',
 'Capítulo',
 'iv',
 'Capítulo',
 'v',
 'Capítulo',
 'vI',
 'Capítulo',
 'vII',
 'CAPÍTULO',
 'VIII',
 'CAPÍTULO',
 'PRIMEIRO',
 'Era',
 'conveniente',
 'ao',
 'romance',
 'que',
 'o',
 'leitor',
 'ficasse',
 'muito',
 'tempo',
 'sem',
 'saber',
 'quem',
 'er

Preparando o Texto
------------------

Na célula abaixo, vamos normalizar os nossos textos trazendo todas as palavras para caixa baixa e abreviando-as de forma a deixar apenas as suas raízes. Neste passo, removeremos também as *stopwords*. Tenha paciência, esta análise vai levar algum tempo...

In [28]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

Vejamos uma amostra do resultado:

In [29]:
textos_limpos[0][150:160]

['uma', 'tal', 'miss', 'doll', 'dev', 'ter', 'poet', 'tennyson', 'cor', 'ler']

Construindo um Índice Invertido
-------------------------------

De posse da nossa lista de termos *normalizados*, podemos agora construir o nosso índice invertido.

In [30]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

Podemos verificar a estrutura interna do nosso índice, procurando por uma palavra:

In [50]:
indice[stemmer.stem("Dollar")]

{0, 3}

In [35]:
print(stemmer.stem('Salarial'))

salarial


Vamos ver o contexto em que a palavra *Salário* ocorre em um dos textos

In [42]:
nltk.Text(WordPunctTokenizer().tokenize(textos[182])).concordance("Salário")

Displaying 2 of 2 matches:
operários que com esse acréscimo de salário proporcionariam às suas famílias ma
s 2 horas da sesta é equivalente ao salário de meio dia , em tais casos abonado


In [96]:
def busca(consulta):
    """
    A construção de uma função de busca é deixada com exercício ao leitor
    """
    pass
    
    

Mas já podemos utilizar nosso índice diretamente com alguns termos e verificar como o mesmo é eficiente. 

In [44]:
%time
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
results

Wall time: 0 ns


{27,
 49,
 61,
 69,
 73,
 84,
 87,
 95,
 122,
 137,
 138,
 141,
 144,
 154,
 155,
 164,
 171,
 189,
 219,
 235}

Para um exame mais preciso do tempo de execução da nossa consulta, podemos usar a mágica do *%%timeit*

In [45]:
%%timeit
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
#results

10000 loops, best of 3: 121 µs per loop
